In [1]:
import numpy as np
import pandas as pd
import glob

from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from matplotlib import pyplot

In [2]:
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

In [3]:
def load_and_transform(load_path, save_path, inVal_unit, outVal, days_of_training = 4, rowLimit = None):
    
    libre_files = glob.glob(f"{load_path}/*")
    print(len(libre_files))
    
    file_n = 1
    
    for f in libre_files:
        print(f)
        series = read_csv(f, header=0, index_col=1, nrows = rowLimit)
        values = series.values

        # values = values[0:20000]

        for j in range(1, (days_of_training + 1)):
            # transform the time series data into supervised learning
            inVal  = inVal_unit*j
            outVal = outVal
            data = series_to_supervised(values, n_in=inVal, n_out = outVal)

            # save out data as supervised data format
            np.savetxt(f"{save_path}/supervised_{file_n}_{inVal}_{outVal}_vals.csv", data, delimiter=",")

        file_n = file_n + 1
        
    return(len(libre_files))
    

In [4]:
def generate_labeled_data(load_path, rowLimit, n_files, train_bins, test_bins, gap_bins, outVal, hypo_thresh, prevelance_ratio):
    
    for i in range(1, n_files+1):
        # print(i)
        sub = pd.read_csv(f"{load_path}/supervised_{i}_{train_bins}_{outVal}_vals.csv",
                     nrows = rowLimit, header = None)

        if i==1:
            concat_output = sub
        else:
            frames = [concat_output, sub]
            concat_output = pd.concat(frames)
            
    print(concat_output.shape)
    
    # take every nth row; where n = test_bins
    all_data = concat_output.iloc[::test_bins, :]

    # divide into train and test splits (cols)
    train_cols = all_data.iloc[:, 0:train_bins]
    test_cols  = all_data.iloc[:, train_bins:train_bins + test_bins + gap_bins]
    
    # create labels
    minvalue = test_cols.min(axis = 1)
    label = np.where(minvalue < hypo_thresh, 1, 0)

    df_X = train_cols.copy()
    df_X['label'] = label
    
    # downsample to a specified ratio of case:control
    df_X_case = df_X[label==1]
    df_X_control = df_X[label==0]

    sample = df_X_control.sample(n=(df_X_case.shape[0] * prevelance_ratio))

    frame = [df_X_case, sample]
    output_frame = pd.concat(frame)
    
    return(output_frame)

In [5]:
# transform a time series dataset into a supervised learning dataset
# return the number of source files (sourceFile_n)
# (ie individuals libre data in the processed raw data folder (load_path))
sourceFile_n = load_and_transform(load_path = "/media/psf/Home/Documents/data/processed_libre",
                  save_path = "/media/psf/Home/Documents/data/libre_as_supervised2",
                  inVal_unit = 96,
                  outVal = 96,
                  days_of_training = 4,
                  rowLimit = 10000)

15
/media/psf/Home/Documents/data/processed_libre/proc_6.csv
/media/psf/Home/Documents/data/processed_libre/proc_7.csv
/media/psf/Home/Documents/data/processed_libre/proc_5.csv
/media/psf/Home/Documents/data/processed_libre/proc_4.csv
/media/psf/Home/Documents/data/processed_libre/proc_1.csv
/media/psf/Home/Documents/data/processed_libre/proc_3.csv
/media/psf/Home/Documents/data/processed_libre/proc_2.csv
/media/psf/Home/Documents/data/processed_libre/proc_14.csv
/media/psf/Home/Documents/data/processed_libre/proc_15.csv
/media/psf/Home/Documents/data/processed_libre/proc_11.csv
/media/psf/Home/Documents/data/processed_libre/proc_10.csv
/media/psf/Home/Documents/data/processed_libre/proc_12.csv
/media/psf/Home/Documents/data/processed_libre/proc_13.csv
/media/psf/Home/Documents/data/processed_libre/proc_9.csv
/media/psf/Home/Documents/data/processed_libre/proc_8.csv


In [5]:
# read in the supervised files and concatenate into a single file
# select by nth row to avoid overfitting / data leakage from one training set into another
# take interval to be duration of test window
# add progressive gaps

In [164]:
# set variables
sourceFile_n = 15 # if all cells not run above ie if starting here need to set this

n_files = sourceFile_n

test_duration_hours = 4
gap_hours = 0

train_bins = 192
test_bins = test_duration_hours * 4
gap_bins = gap_hours * 4

hypo_thresh = 3

prevelance_ratio = 1

# concatenate the data files, add label
data_lab = generate_labeled_data("/media/psf/Home/Documents/data/libre_as_supervised",
                                 None,
                                 n_files,
                                 train_bins,
                                 test_bins,
                                 gap_bins,
                                 96,
                                 hypo_thresh,
                                 prevelance_ratio)

# concatenate the data files, add label
data_lab['id'] = np.arange(len(data_lab))

# save out
save_path = '/media/psf/Home/Documents/data/libre_labeled'
np.savetxt(f"{save_path}/labeledData_{train_bins}_trainBins_{test_bins}_testBins_{gap_bins}_gapBins_{hypo_thresh}_hypoThresh_{prevelance_ratio}_prevalenceRatio.csv", data_lab, delimiter=",")

(1104107, 288)


In [169]:
# generate the features for analysis - by measurement and by day
# convert to long format

for i in range(0, data_lab.shape[0] + 1):
    
    if i%100 == 0:
        print(i / data_lab.shape[0])
        
    sub = data_lab[data_lab['id'] == i]
    
    label = sub['label']
    sub   = sub.drop(['id', 'label'], axis=1)
    
    new   = sub.transpose()
    new.columns = ['glu']
    
    new['timestep'] = np.arange(len(new))
    new['id']       = i
    new['label']    = 0
    new.assign(label = label)
    
    for j in range(0, int(new.shape[0] / 96)):
        j = j+1
        c = np.repeat(j, 96)
        if j==1:
            cc = c
        else:
            cc = np.concatenate((cc, c))
    
    new['day_label'] = cc
    
    if i==0:
        newcat=new
    else:
        frame = [newcat, new]
        newcat = pd.concat(frame)
    
 #    if i==1000:
  #       break

0.0
0.02074688796680498
0.04149377593360996
0.06224066390041494
0.08298755186721991
0.1037344398340249
0.12448132780082988
0.14522821576763487
0.16597510373443983
0.18672199170124482
0.2074688796680498
0.22821576763485477
0.24896265560165975
0.2697095435684647
0.29045643153526973
0.3112033195020747
0.33195020746887965
0.35269709543568467
0.37344398340248963
0.3941908713692946
0.4149377593360996
0.43568464730290457
0.45643153526970953
0.47717842323651455
0.4979253112033195
0.5186721991701245
0.5394190871369294
0.5601659751037344
0.5809128630705395
0.6016597510373444
0.6224066390041494
0.6431535269709544
0.6639004149377593
0.6846473029045643
0.7053941908713693
0.7261410788381742
0.7468879668049793
0.7676348547717843
0.7883817427385892
0.8091286307053942
0.8298755186721992
0.8506224066390041
0.8713692946058091
0.8921161825726142
0.9128630705394191
0.9336099585062241
0.9543568464730291
0.975103734439834
0.995850622406639


ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [172]:
df = data_lab
df["id"] = df.index
pd.wide_to_long(df, i="id", j="label")

TypeError: wide_to_long() missing 1 required positional argument: 'stubnames'

In [103]:
for j in range(1, days):
    c = np.repeat(j, 96)
    print(c)


TypeError: 'float' object cannot be interpreted as an integer

In [96]:
c

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [190]:
for i in range(1, n_files+1):
    # print(i)
    sub = pd.read_csv(f"/media/psf/Home/Documents/data/libre_as_supervised/supervised_{i}_{train_bins}_{outVal}_vals.csv",
                 nrows = None, header = None)
    
    if i==1:
        concat_output = sub
    else:
        frames = [concat_output, sub]
        concat_output = pd.concat(frames)

In [191]:
concat_output.shape

(1104107, 288)

In [192]:
# take every nth row; where n = test_bins
all_data = concat_output.iloc[::test_bins, :]

# divide into train and test splits (cols)
train_cols = all_data.iloc[:, 0:train_bins]
test_cols  = all_data.iloc[:, train_bins:train_bins + test_bins + gap_bins]

In [193]:
all_data.shape

(69007, 288)

In [196]:
# create labels
minvalue = test_cols.min(axis = 1)
label = np.where(minvalue < hypo_thresh, 1, 0)

df_X = train_cols.copy()
df_X['label'] = label

In [197]:
df_X.shape

(69007, 193)

In [200]:
# downsample to a specified ratio of case:control
df_X_case = df_X[label==1]
df_X_control = df_X[label==0]

sample = df_X_control.sample(n=(df_X_case.shape[0] * prevelance_ratio))

frame = [df_X_case, sample]
output_frame = pd.concat(frame)

In [201]:
df_X_case.shape

(2410, 193)

In [203]:
output_frame.shape

(12050, 193)